In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


In [2]:
from datasets import load_dataset
train_data = load_dataset('nsmc',split='train[:80%]')
val_data = load_dataset('nsmc',split='train[-20%:]')
test_data = load_dataset('nsmc',split='test')

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


In [3]:
import re
def preprocess_sentence(sentence):
    # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣.,?!]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [4]:
print(sum(train_data['label'])/ len(train_data))
print(sum(val_data['label'])/ len(val_data))
print(sum(test_data['label'])/ len(test_data))

0.49924166666666664
0.4972666666666667
0.50346


In [5]:
l = 0
for s in train_data['document']:
    l += len(s)
l/len(train_data)

35.235866666666666

In [6]:
train_data = train_data.map(lambda example: {'document': preprocess_sentence(example['document'])})
val_data = val_data.map(lambda example: {'document': preprocess_sentence(example['document'])})
test_data = test_data.map(lambda example: {'document': preprocess_sentence(example['document'])})

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/30000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

# Tokenizer 불러오기

In [7]:
from transformers import BertForSequenceClassification, BertTokenizer
HUGGINGFACE_MODEL_PATH = "klue/bert-base"
model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=2)
config = model.config
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [8]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation = True,
        
        padding = True,
        return_token_type_ids = False,
        max_length=40
        )

In [9]:
train =train_data.map(transform, batched=True)
val = val_data.map(transform, batched=True)
test = test_data.map(transform, batched=True)

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [10]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/0326'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,
    lr_scheduler_type="cosine"# weight decay
)

In [11]:
from datasets import load_metric
metric = load_metric('accuracy')
#metric = load_metric('glue',accuracy')
def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [12]:
# from transformers import AdamW, get_scheduler
# num_training_steps = len(train) // training_arguments.per_device_train_batch_size * training_arguments.num_train_epochs
# optimizer = AdamW(model.parameters(), lr=training_arguments.learning_rate)
# # Initialize the linear learning rate scheduler
# lr_scheduler = get_scheduler(
#     "cosine",
#     optimizer = optimizer,
#     num_warmup_steps=100,
#     num_training_steps=num_training_steps,
    
# )

In [13]:
trainer = Trainer(
    model= model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,    # training dataset
    eval_dataset=val,       # evaluation dataset
    compute_metrics=compute_metrics,
    #lr_scheduler=lr_scheduler,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45000


Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-500
Configuration saved in /aiffel/aiffel/0326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-1000
Configuration saved in /aiffel/aiffel/0326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-1500
Configuration saved in /aiffel/aiffel/0326/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-2000
Configuration saved in /aiffel/aiffel/0326/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-2500
Configuration saved in /aiffel/aiffel/0326/checkpoint-2500/config.json
Model weights saved in /a

TrainOutput(global_step=45000, training_loss=0.245214357164171, metrics={'train_runtime': 4957.6095, 'train_samples_per_second': 72.616, 'train_steps_per_second': 9.077, 'total_flos': 7399998432000000.0, 'train_loss': 0.245214357164171, 'epoch': 3.0})

group_by_length (bool, optional, defaults to False

— Whether or not to group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient). Only useful if applying dynamic padding.

The main idea behind the 2 optimizations is to avoid as much unused computation as possible:

Dynamic padding: limit the number of added pad tokens to reach the length of the longest sequence of each mini batch instead of a fixed value set for the whole train set. Because the number of added tokens changes across mini batches, we call it “dynamic” padding ;

Uniform length batching: we push the logic further by generating batches made of similar length sequences, so we avoid extreme cases where most sequences in the mini batch are short and we are required to add lots of pad tokens to each of them because 1 sequence of the same mini batch is very long.

In [14]:
trainer.evaluate(test)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.47963955998420715,
 'eval_accuracy': 0.90042,
 'eval_runtime': 127.5258,
 'eval_samples_per_second': 392.077,
 'eval_steps_per_second': 49.01,
 'epoch': 3.0}

# Bucketing 적용

In [15]:
from transformers import BertForSequenceClassification, BertTokenizer
HUGGINGFACE_MODEL_PATH = "klue/bert-base"
model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=2)
config = model.config
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    group_by_length = True,
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,
    lr_scheduler_type="cosine"# weight decay
)

trainer = Trainer(
    model= model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    data_collator = data_collator,
    train_dataset=train,    # training dataset
    eval_dataset=val,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45000


Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-500
Configuration saved in /aiffel/aiffel/0326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-1000
Configuration saved in /aiffel/aiffel/0326/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-1500
Configuration saved in /aiffel/aiffel/0326/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-2000
Configuration saved in /aiffel/aiffel/0326/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-2500
Configuration saved in /aiffel/aiffel/0326/checkpoint-2500/config.json
Model weights saved in /a

TrainOutput(global_step=45000, training_loss=0.24618472112019857, metrics={'train_runtime': 6100.9631, 'train_samples_per_second': 59.007, 'train_steps_per_second': 7.376, 'total_flos': 7399998432000000.0, 'train_loss': 0.24618472112019857, 'epoch': 3.0})

In [17]:
trainer.evaluate(test)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.4826755225658417,
 'eval_accuracy': 0.89848,
 'eval_runtime': 127.2831,
 'eval_samples_per_second': 392.825,
 'eval_steps_per_second': 49.103,
 'epoch': 3.0}

## Bucketing 유무의 비교

Bucketing이란 시퀀스의 길이별로 정렬하고 버킷에 그룹화를 진행한다. 각 버킷 내의 시퀀스는 유사한 길이를 가지고 더 효율적인 배치와 최소화된 패딩을 가진다. HuggingFace의 DatacollatorWithPadding을 통해 진행한다.

동적 패딩이란 각 배치 내의 가장 긴 시퀀스의 길이에 맞게 시퀀스를 패딩하는 것이다. 이로 모델은 각 배치에 필요한 만큼의 패딩만 처리되어, 모델의 효율성을 증가 시킬수 있다.

- Without Bucketing :
    - 'train_runtime': 561.0603,
    - 'train_samples_per_second': 8.021,
    - 'train_steps_per_second': 1.005,
    - 'eval_accuracy': 0.8532,
    - 'eval_runtime': 164.3683,
    - 'eval_samples_per_second': 30.419,
    - 'eval_steps_per_second': 3.802,
- With Bucketing :
    - 'train_runtime': 575.8021,
    - 'train_samples_per_second': 7.815,
    - 'train_steps_per_second': 0.98,
    - 'eval_accuracy': 0.8528,
    - 'eval_runtime': 164.2709,
    - 'eval_samples_per_second': 30.438,
    - 'eval_steps_per_second': 3.805,
    
- Without Bucketing :
    - 'train_runtime': 4957.6095,
    - 'train_samples_per_second': 72.616,
    - 'train_steps_per_second': 9.077,
    - eval_accuracy': 0.90042,
    - 'eval_runtime': 127.5258,
    - 'eval_samples_per_second': 392.077,
- With Bucketing :
    - 'train_runtime': 6100.9631,
    - 'train_samples_per_second': 59.007, 
    - 'train_steps_per_second': 7.376,
    - 'eval_accuracy': 0.89848,
    - 'eval_runtime': 127.2831,
    - 'eval_samples_per_second': 392.825,
    - 'eval_steps_per_second': 49.103,